In [72]:
import pandas as pd
import requests
import multiprocessing
train_images_list = pd.read_csv("gs://uga-dsp/project3/train.txt", header=None).to_numpy().squeeze()
test_images_list = pd.read_csv("gs://uga-dsp/project3/test.txt", header=None).to_numpy().squeeze()


In [73]:
import numpy as np

In [74]:
# all_files = np.append(train_images_list, test_images_list)
train_data_files = train_images_list + '.tar'
test_data_files = test_images_list + '.tar'
mask_files = train_images_list + '.png'

In [75]:
import urllib3
http = urllib3.PoolManager()

base_url_masks = 'http://storage.googleapis.com/uga-dsp/project3/masks/'
base_url_data = 'http://storage.googleapis.com/uga-dsp/project3/data/'

In [76]:
def download_train_data(file_name):
#     print(file_name)
    filedata = http.request('GET', base_url_data + file_name).data
    with open('/home/vraskap3/datas/train/' + file_name, 'wb') as f:
        f.write(filedata)
    tfile = tarfile.open(f'/home/vraskap3/datas/train/{file_name}')
    tfile.extractall(path='/home/vraskap3/datas/train')
    return file_name

In [77]:
def download_test_data(file_name):
    filedata = http.request('GET', base_url_data + file_name).data
    with open('/home/vraskap3/datas/test/' + file_name, 'wb') as f:
        f.write(filedata)
    tfile = tarfile.open(f'/home/vraskap3/datas/test/{file_name}')
    tfile.extractall(path='/home/vraskap3/datas/test/')
    return file_name

In [78]:
def download_masks(file_name):
    filedata = http.request('GET', base_url_masks + file_name).data
    with open('/home/vraskap3/datas/masks/' + file_name, 'wb') as f:
        f.write(filedata)
    return file_name

In [79]:
from multiprocessing import Pool, cpu_count
from functools import partial
from io import BytesIO

def downloader(func, file_list):
    num_cores = multiprocessing.cpu_count()

    pool = Pool(num_cores)
    download_func = partial(func)
    results = pool.map(download_func, file_list)
    pool.close()
    pool.join()

In [80]:
print("Downloading masks")
downloader(download_masks, mask_files)
print("Downloading data")
downloader(download_train_data, train_data_files)
print("Downloading data")
downloader(download_test_data, test_data_files)


In [81]:
import tarfile